In [2]:
import pandas as pd

df_jp = pd.read_excel('../data/raw/japan_floods_raw.xlsx', engine='openpyxl')
df_jp.head()

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
0,2004-0339-JPN,No,nat-hyd-flo-fla,Natural,Hydrological,Flood,Flash flood,GLIDE:FL-2004-000052,NaN,JPN,...,NaN,NaN,279000.0,463351.0,1950000.0,3238472.0,60.213580,"[{""adm1_code"":1657,""adm1_name"":""Hukusima""},{""a...",2004-08-02,2025-06-25
1,1972-0147-JPN,Yes,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,NaN,JPN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.330618,NaN,2003-07-01,2023-09-25
2,1998-0248-JPN,Yes,nat-hyd-flo-coa,Natural,Hydrological,Flood,Coastal flood,NaN,NaN,JPN,...,NaN,NaN,NaN,NaN,NaN,NaN,51.964968,NaN,2003-07-01,2023-09-25
3,2005-0394-JPN,No,nat-hyd-flo-riv,Natural,Hydrological,Flood,Riverine flood,GLIDE:FL-2005-000101,NaN,JPN,...,NaN,NaN,NaN,NaN,NaN,NaN,62.256479,"[{""adm1_code"":1678,""adm1_name"":""Niigata""}]",2005-07-25,2025-06-25
4,1995-0146-JPN,Yes,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,NaN,JPN,...,NaN,NaN,NaN,NaN,74300.0,152950.0,48.577848,NaN,2003-07-01,2023-09-25


In [3]:
df_jp.columns

Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'Entry Date', 'Last Update'],
      dtype='object')

In [4]:
# 1. Keep only floods and years 1967‑2023
df_jp = df_jp[
    (df_jp['Disaster Type'].str.lower() == 'flood') &
    (df_jp['Start Year'].between(1967, 2023))
]

In [5]:
# 2. Build a real start_date column
df_jp['start_date'] = pd.to_datetime(
    dict(year=df_jp['Start Year'],
         month=df_jp['Start Month'].fillna(1).astype(int),
         day=df_jp['Start Day'].fillna(1).astype(int)),
    errors='coerce'
)

In [6]:
# 3. Rename core columns
df_jp = df_jp.rename(columns={
    'Start Year'       : 'year',
    'Admin Units'      : 'state',          # or 'Subregion'
    'Total Deaths'     : 'human_fatality',
    'Total Affected'   : 'affected',
    "Total Damage ('000 US$')" : 'damage_usd_thousands'
})


In [8]:
df_jp.columns.tolist()

['DisNo.',
 'Historic',
 'Classification Key',
 'Disaster Group',
 'Disaster Subgroup',
 'Disaster Type',
 'Disaster Subtype',
 'External IDs',
 'Event Name',
 'ISO',
 'Country',
 'Subregion',
 'Region',
 'Location',
 'Origin',
 'Associated Types',
 'OFDA/BHA Response',
 'Appeal',
 'Declaration',
 "AID Contribution ('000 US$)",
 'Magnitude',
 'Magnitude Scale',
 'Latitude',
 'Longitude',
 'River Basin',
 'year',
 'Start Month',
 'Start Day',
 'End Year',
 'End Month',
 'End Day',
 'human_fatality',
 'No. Injured',
 'No. Affected',
 'No. Homeless',
 'affected',
 "Reconstruction Costs ('000 US$)",
 "Reconstruction Costs, Adjusted ('000 US$)",
 "Insured Damage ('000 US$)",
 "Insured Damage, Adjusted ('000 US$)",
 "Total Damage ('000 US$)",
 "Total Damage, Adjusted ('000 US$)",
 'CPI',
 'state',
 'Entry Date',
 'Last Update',
 'start_date']

In [9]:
# Rename columns to standard names we use in the project
rename_map = {
    'Start Year': 'year',
    'start_date': 'start_date',
    'state': 'state',
    'Total Deaths': 'human_fatality',
    'Total Affected': 'affected',
    "Total Damages ('000 US$')": 'damage_usd_thousands'
}

df_jp = df_jp.rename(columns={k: v for k, v in rename_map.items() if k in df_jp.columns})

# Now keep only the renamed columns
keep_cols = ['year', 'start_date', 'state', 'human_fatality', 'affected']
if 'damage_usd_thousands' in df_jp.columns:
    keep_cols.append('damage_usd_thousands')

df_jp = df_jp[keep_cols]

# Fill missing values
df_jp['human_fatality'] = df_jp['human_fatality'].fillna(0).astype(int)
df_jp['affected'] = df_jp['affected'].fillna(0).astype(int)

if 'damage_usd_thousands' in df_jp.columns:
    df_jp['damage_usd_thousands'] = df_jp['damage_usd_thousands'].fillna(0).astype(float)


In [10]:
df_jp.to_csv('../data/clean/japan_floods_cleaned.csv', index=False)

In [11]:
df_jp.head()

,year,start_date,state,human_fatality,affected
0,2004,2004-07-12,"[{""adm1_code"":1657,""adm1_name"":""Hukusima""},{""a...",21,25807
1,1972,1972-07-03,NaN,458,0
2,1998,1998-08-26,NaN,16,24033
3,2005,2005-06-28,"[{""adm1_code"":1678,""adm1_name"":""Niigata""}]",0,900
4,1995,1995-07-01,NaN,0,12000


In [12]:
import ast

def extract_state_names(cell):
    if pd.isna(cell):
        return None
    try:
        data = ast.literal_eval(cell)
        if isinstance(data, list):
            return ', '.join([entry.get('adm1_name', '') for entry in data if 'adm1_name' in entry])
        return None
    except Exception:
        return None

df_jp['state'] = df_jp['state'].apply(extract_state_names)

In [13]:
df_jp[['state']].dropna().head(10)

,state
0,"Hukusima, Niigata"
3,Niigata
5,"Hukuoka, Nagasaki"
6,"Hirosima, Nagasaki"
8,Hukusima
9,
12,"Miyagi, Totigi"
17,Kumamoto
18,"Ehime, Hukuoka, Kagawa, Kagosima, Kooti, Kumam..."
20,"Hukuoka, Hirosima, Kagosima, Kumamoto, Miyazak..."


In [14]:
df_jp.to_csv('../data/clean/japan_floods_cleaned.csv', index=False)